In [ ]:
#works 
import pandas as pd

df = pd.read_excel('Japanese Stocks.xlsx')

result_df = pd.DataFrame(columns=['Name1', 'Name2', 'Correlation', 'Edge'])

# Gets unique company names
company_names = df['Name'].unique()

columns_to_include = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

for i in range(len(company_names)):
    for j in range(i + 1, len(company_names)):
        name1 = company_names[i]
        name2 = company_names[j]

        data_name1 = df[df['Name'] == name1][['Date'] + columns_to_include].set_index('Date')
        data_name2 = df[df['Name'] == name2][['Date'] + columns_to_include].set_index('Date')

        merged_data = pd.merge(data_name1, data_name2, left_index=True, right_index=True, suffixes=('_' + str(name1), '_' + str(name2))
)

        correlation_matrix = merged_data.corr()
        correlation = correlation_matrix.loc['Close_' + str(name1), 'Close_' + str(name2)]


        edge = 'Yes' if correlation > 0.9 else 'No'

        
        result_df = pd.concat([result_df, pd.DataFrame({'Name1': [name1], 'Name2': [name2], 'Correlation': [correlation], 'Edge': [edge]})], ignore_index=True)

result_df.to_excel('correlation_results_JPN.xlsx', index=False)

In [ ]:
import pandas as pd
import igraph as ig
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_excel('correlation_results_JPN.xlsx')

g = ig.Graph.TupleList(
    df[df['Edge'] == 'Yes'][['Name1', 'Name2', 'Correlation']].itertuples(index=False),
    directed=False,
    weights='Correlation'  
)


sample_edge = g.es[0]  
#print(sample_edge.attributes())

g.simplify(multiple=True, loops=True)


for vertex in g.vs:
    outgoing_edges = g.es.select(_source=vertex.index)
    total_weight = sum(edge.attributes().get('Correlation', 0) for edge in outgoing_edges)
    for edge in outgoing_edges:
        if 'Correlation' in edge.attributes():
            edge['Correlation'] /= total_weight


id_to_letter = {i: chr(65 + i) for i in range(g.vcount())}
vertex_labels = [f"{id_to_letter[id]}" for id in range(g.vcount())]


layout = g.layout_fruchterman_reingold() 
fig = make_subplots(rows=1, cols=1)


for edge in g.get_edgelist():
    x0, y0 = layout[edge[0]]
    x1, y1 = layout[edge[1]]
    fig.add_scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color='gray', width=1))


fig.add_scatter(x=[coord[0] for coord in layout], y=[coord[1] for coord in layout],
                mode='markers',
                marker=dict(size=10, color=[ "black" for v in range(g.vcount())]),
                text=vertex_labels, showlegend=False)


fig.update_layout(
    width=1200, 
    height=800, 
    margin=dict(l=0, r=0, b=0, t=0),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

fig.add_annotation(
    x=0,
    y=0,
    xref="paper",
    yref="paper",
    text=f"Number of Nodes: {g.vcount()}",
    showarrow=False,
    font=dict(size=14),
)

fig.show()
